# Geometry Tutorial

This is an introductory walkthrough of the symbolic 2D and 3D [geometry package](../api/symforce.geo.html) in symforce.

In [ ]:
# Setup
import symforce
symforce.set_backend('sympy')
symforce.set_log_level('warning')

from symforce.notebook_util import display, display_code, display_code_file

In [ ]:
from symforce import sympy as sm
from symforce import geo

Let's create a 3D rotation around the Z axis about a variable number of degrees:

In [ ]:
angle = sm.Symbol('theta')
rot = geo.Rot3.from_axis_angle(
    axis=geo.V3(0, 0, 1),
    angle=angle * (sm.pi / 180)
)
display(rot)

All geometric types can be serialized to a list of scalar expressions. In this case, into the four values of the underlying quaternion:

In [ ]:
elements = rot.to_storage()
display(elements)

They can also be constructed back from the serialized values:

In [ ]:
rot2 = geo.Rot3.from_storage(elements)
assert rot == rot2

Let's rotate a point:

In [ ]:
point = geo.V3().symbolic('P')
display(point)

In [ ]:
rotated_point = rot * point
display(rotated_point)

Note that we can get a good feeling for the expression by looking at the symbolic representation. Let's plug in some values for the angle:

In [ ]:
display([
    rotated_point.subs(angle, 0),
    rotated_point.subs(angle, 45),
    rotated_point.subs(angle, 90)
])

Another example, reversing a pose transform to go from from world to body coordinates. Note we're using a naming convention where the destination frame is prefixed and the source frame is suffixed, so `world_T_body` transforms from the body frame to the world frame. `T` is a pose transform, `R` is a rotation, and `t` is a translation:

In [ ]:
world_T_body = geo.Pose3(
    R=geo.Rot3.symbolic('q'),
    t=geo.V3().symbolic('t')
)
display(world_T_body)
display(world_T_body.to_storage())

In [ ]:
world_t_point = geo.V3().symbolic('P')
body_t_point = world_T_body.inverse() * world_t_point
display(body_t_point)

## Concepts

SymForce uses [concepts](https://en.wikipedia.org/wiki/Concept_(generic_programming)) as an underlying mechanism. A concept is a specification of supported operations, including syntax and semantics, but does not require a subtype relationship. This means that a set of heterogenous types can be operated on in a homogenous way, ie types that are external and don't share a base class, like Python floats treated as scalars.

There are three core concepts for geometric types and scalars, each of which is a superset of the previous. The core routines use these ops interfaces rather than calling methods on types directly. The API docs provide much more detail and each op is tested on each type, but examples are given here:

In [ ]:
from symforce.ops import StorageOps, GroupOps, LieGroupOps

[StorageOps](../api/symforce.ops.storage_ops.html): **Data type that can be serialized to and from a vector of scalar quantities.**

Methods: `.storage_dim()`, `.to_storage()`, `.from_storage()`

Storage operations are used extensively for marshalling and for operating on each scalar in a type.

In [ ]:
# Elements in a Pose3 (4 quaternion + 3 position)
display(StorageOps.storage_dim(geo.Pose3))

In [ ]:
# Elements in a scalar
display(StorageOps.storage_dim(float))

In [ ]:
# Serialize scalar
display(StorageOps.to_storage(5))

In [ ]:
# Serialize vector/matrix
display(StorageOps.to_storage(geo.V3(sm.Symbol('x'), 5.2, sm.sqrt(5) + point[1])))

In [ ]:
# Serialize geometric type
display(StorageOps.to_storage(world_T_body))

In [ ]:
# Reconstruct
display(StorageOps.from_storage(geo.Pose2, [1, 0, 3, 4]))

[GroupOps](../api/symforce.ops.group_ops.html): **Mathematical group that implements closure, associativity, identity and invertibility.**

Methods: `.identity()`, `.inverse()`, `.compose()`

Group operations provide the core methods to compare and combine types.

In [ ]:
# Identity of a pose
display(GroupOps.identity(geo.Pose3))

In [ ]:
# Identity of a scalar (under addition)
display(GroupOps.identity(float))

In [ ]:
# Inverse of a vector
display(GroupOps.inverse(geo.V3(1.2, -3, 2)).T)

In [ ]:
# Compose two vectors (under addition)
display(GroupOps.compose(geo.V2(1, 2), geo.V2(3, -5)))

In [ ]:
# Compose a rotation and its inverse to get identity
display(GroupOps.compose(rot, rot.inverse()).to_storage())

[LieGroupOps](../api/symforce.ops.lie_group_ops.html): **Group that is also a differentiable manifold, such that calculus applies.**

Methods: `.tangent_dim()`, `.from_tangent()`, `to_tangent()`

Lie group operations provide the core methods for nonlinear optimization.  
Familiarity is not expected for all users, but learning is encouraged!

In [ ]:
# Underlying dimension of a 3D rotation
display(LieGroupOps.tangent_dim(geo.Rot3))

In [ ]:
# Exponential map for a 2D rotation
rot2 = LieGroupOps.from_tangent(geo.Rot2, [angle])
display(rot2.to_rotation_matrix())

In [ ]:
# Skew symmetric for a 3D rotation
display(geo.M(geo.Rot3().hat([1, 2, 3])))

In [ ]:
# Logarithmic map of the rotation about the X axis
display(LieGroupOps.to_tangent(rot))

In [ ]:
# Exponential map of a vector type is a no-op
display(LieGroupOps.from_tangent(geo.V5(), [1, 2, 3, 4, 5]).T)

Using symbolic geometric types and concepts is already very powerful for development and analysis of robotics, but operating on symbolic objects at runtime is much too slow for most applications. However, symbolic expressions can be beautifully set to fast runtime code.